In [1]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

liste_saison = ["2023_2024", "2022_2023", "2021_2022", "2020_2021"]

saison = liste_saison[0]

event_import = pd.read_json(f"../../Data_file/Heatmap SB/{saison}.json")

In [48]:
event = event_import[~event_import.location.isna()]

event = event.sort_values(by = ["match_id", "index"])
event["index"] = range(len(event))

centre = event[(event.pass_cross == 1) & (event.pass_type != "Corner")]

centre = pd.DataFrame({"attacking_team" : [i for i in centre.team for _ in range (6)],
                    "centre_id" : [i for i in range (1, len(centre) + 1) for _ in range (6)],
                    "match_id" : [i for i in centre.match_id for _ in range (6)],
                    "index" : [j + i for j in centre["index"] for i in range (6)]})

liste_centre_drop = centre.centre_id[centre["index"].duplicated()].unique() - 1
centre = centre.drop_duplicates("index", keep = "last")

df = pd.merge(centre, event, on = ["index", "match_id"])

df["But"] = "Non"
df["Tireur"] = ""

mask_shot = (df.type == "Shot") & (df.attacking_team == df.team)

df_shot = df[mask_shot]
df_loc_shot = pd.DataFrame(df_shot.location.tolist(), index = df_shot.index)[[0, 1]]
df_loc_shot_end = pd.DataFrame(df_shot.shot_end_location.tolist(), index = df_shot.index)

df.loc[df_shot.index, "Tireur"] = df_shot.player

df.loc[mask_shot, "x"] = df_loc_shot[0]
df.loc[mask_shot, "y"] = df_loc_shot[1]

df.loc[mask_shot, "x_end"] = df_loc_shot_end[0]
df.loc[mask_shot, "y_end"] = df_loc_shot_end[1]
df.loc[mask_shot, "z_end"] = df_loc_shot_end[2]

df_goal = df[((df.shot_outcome == "Goal") | (df.type == "Own Goal For")) & (df.attacking_team == df.team)]

df.loc[(df.centre_id.isin(df_goal.centre_id)) & (df.pass_cross == 1), "But"] = "Oui"
df.loc[df_goal.index, "Tireur"] = df_goal.player


mask_centre = df.pass_cross == 1

df_loc_centre = pd.DataFrame(df.loc[mask_centre, "location"].tolist(), index = df.loc[mask_centre].index)
df.loc[mask_centre, "x"] = df_loc_centre[0]
df.loc[mask_centre, "y"] = df_loc_centre[1]

df_loc_centre_end = pd.DataFrame(df.loc[mask_centre, "pass_end_location"].tolist(),
    index = df.loc[mask_centre].index)
df.loc[mask_centre, "x_end"] = df_loc_centre_end[0]
df.loc[mask_centre, "y_end"] = df_loc_centre_end[1]

df_centre_drop = df[df.centre_id.isin(liste_centre_drop)]
df.drop(df_centre_drop[df_centre_drop.But == "Non"].index, inplace = True)

df_centreur = df.groupby("centre_id").head(1)[['player', 'centre_id']]
df = pd.merge(df, df_centreur, on = "centre_id").set_index(df.index)

df = df[["pass_cross", "match_id", "attacking_team", "shot_outcome", "But", "Tireur", "player_y", "minute", "pass_body_part", "x", "y", "x_end", "centre_id",
            "y_end", "z_end"]]
df.rename(axis = 1, mapper = {"pass_cross" : "Centre", "attacking_team" : "Équipe attaquante", "player_y" : "Centreur",
                                "pass_body_part" : "Partie du corps"}, inplace = True)

In [53]:
sum(df.groupby("centre_id").Centre.sum() == 1)

8919

In [105]:
df.groupby(["match_id", "centre_id"]).head(1)[df.groupby(["match_id", "centre_id"]).head(1).pass_cross != 1]

,level_0,attacking_team,centre_id,shot_type,shot_outcome,type,match_id,period,possession,location,team,pass_cross,pass_type,index,pass_end_location,minute,player,shot_end_location,pass_body_part
850,23923,Laval,142,None,None,Pass,3896590,1,2,"[61.0, 40.1]",Paris FC,NaN,Kick Off,5,"[60.0, 39.3]",0,Pierre-Yves Hamel,None,Right Foot
2134,60441,Laval,356,None,None,Pass,3896600,1,2,"[61.0, 40.1]",Paris FC,NaN,Kick Off,5,"[55.3, 37.9]",0,Pierre-Yves Hamel,None,Right Foot
3646,106119,Grenoble Foot,608,None,None,Pass,3896613,1,2,"[60.0, 40.0]",FC Annecy,NaN,Kick Off,5,"[54.5, 34.5]",0,Warren Caddy,None,Right Foot
5458,155216,Pau,910,None,None,Pass,3896627,1,2,"[61.0, 40.1]",Paris FC,NaN,Kick Off,5,"[57.7, 37.7]",0,Pierre-Yves Hamel,None,Right Foot
5872,165667,Caen,979,None,None,Pass,3896630,1,2,"[61.0, 40.1]",Troyes,NaN,Kick Off,5,"[46.6, 43.2]",0,Nicolas de Roussel de Préville,None,Right Foot
10463,276767,Paris FC,1744,None,None,Pass,3896661,1,2,"[60.0, 40.0]",Angers,NaN,Kick Off,5,"[49.5, 35.2]",0,Himad Abdelli,None,Right Foot
13216,338182,Valenciennes,2203,None,None,Pass,3896679,1,2,"[60.0, 40.0]",Guingamp,NaN,Kick Off,5,"[39.8, 35.0]",0,Amine El Ouazzani,None,Left Foot
14500,370228,Saint-Étienne,2417,None,None,Pass,3896688,1,2,"[61.0, 40.1]",Auxerre,NaN,Kick Off,5,"[50.7, 42.2]",0,Florian Aye,None,Right Foot
19676,494797,Rodez,3280,None,None,Pass,3896723,1,2,"[60.0, 40.0]",Troyes,NaN,Kick Off,5,"[35.3, 41.0]",0,Alexis Lefebvre,None,Right Foot
22805,580398,Grenoble Foot,3801,None,None,Pass,3896747,1,2,"[61.0, 40.1]",Guingamp,NaN,Kick Off,5,"[36.5, 42.9]",0,Amine El Ouazzani,None,Left Foot


In [30]:
pd.merge(df, df.groupby("centre_id").head(1).player, left_index = True, right_index = True).player_y.head(20)

level_0
14                       Yacine Bammou
105                         Riad Nouri
163     Ahmad Toure Ngouyamsa Nounchil
322                     Cyrille Bayala
369                         Riad Nouri
423                  Vincent Marchetti
435                    Stephen Quemper
446                    Stephen Quemper
480                  Vincent Marchetti
603                         Riad Nouri
680                    Stephen Quemper
1039                   Stephen Quemper
1225                  Mohamed Youssouf
1451                    Cyrille Bayala
1626              Abdel Hakim Abdallah
1641                    Cyrille Bayala
1694              Abdel Hakim Abdallah
2374                   Kilian Corredor
2468              Abdel Hakim Abdallah
2574                   Stephen Quemper
Name: player_y, dtype: object

In [183]:
df[(df.Tireur != "") & (df.shot_outcome.isna())][["x_end", "y_end", "z_end"]]

,x_end,y_end,z_end
level_0,,,
181968,NaN,NaN,NaN
233993,NaN,NaN,NaN
259594,NaN,NaN,NaN
340939,NaN,NaN,NaN
427763,NaN,NaN,NaN
682690,NaN,NaN,NaN
1165383,NaN,NaN,NaN
1191512,NaN,NaN,NaN
1223861,NaN,NaN,NaN


In [167]:
df2 = df.copy()
df_centreur = df.groupby("centre_id").head(1).player
df2.player = [i for i in df_centreur for _ in range (6)]
df2.player.head(50)

level_0
14                      Yacine Bammou
15                      Yacine Bammou
16                      Yacine Bammou
17                      Yacine Bammou
18                      Yacine Bammou
19                      Yacine Bammou
105                        Riad Nouri
106                        Riad Nouri
107                        Riad Nouri
108                        Riad Nouri
109                        Riad Nouri
110                        Riad Nouri
163    Ahmad Toure Ngouyamsa Nounchil
164    Ahmad Toure Ngouyamsa Nounchil
165    Ahmad Toure Ngouyamsa Nounchil
166    Ahmad Toure Ngouyamsa Nounchil
167    Ahmad Toure Ngouyamsa Nounchil
168    Ahmad Toure Ngouyamsa Nounchil
322                    Cyrille Bayala
323                    Cyrille Bayala
324                    Cyrille Bayala
325                    Cyrille Bayala
326                    Cyrille Bayala
327                    Cyrille Bayala
369                        Riad Nouri
370                        Riad Nouri
371 

Index(['centre 79', 'centre 101', 'centre 142', 'centre 142', 'centre 142',
       'centre 172', 'centre 180', 'centre 180', 'centre 180', 'centre 229',
       ...
       'centre 8962', 'centre 8962', 'centre 8973', 'centre 9005',
       'centre 9005', 'centre 9005', 'centre 9063', 'centre 9063',
       'centre 9063', 'centre 9080'],
      dtype='object', name='centre_id', length=427)

In [105]:
event = event_import[~event_import.location.isna()]

event.sort_values(by = ["match_id", "index"], inplace = True)

event.index = range(len(event))

centre = event[(event.pass_cross == 1) & (event.pass_type != "Corner")]

centre = pd.DataFrame({"attacking_team" : [i for i in centre.team for _ in range (6)],
                       "centre_id" : [i for i in range (1, len(centre) + 1) for _ in range (6)]},
                       index = [j + i for j in centre.index for i in range(6)])

df = pd.concat([event.loc[centre.index], centre], axis = 1).reset_index().set_index(["centre_id", "attacking_team"])

# L_drop = (df[df.level_0.duplicated()].index.get_level_values(0) - 1).unique()
# df.drop_duplicates("level_0", keep = "last", inplace = True)

# df["But"] = 0

# df_shot = df[(df.type == "Shot") & (df.index.get_level_values(1) == df.team)]

# df_shot_player = df_shot.groupby(level = [0, 1]).player.transform(lambda x: ', '.join(x))
# df_shot_player = df_shot_player.reset_index().drop_duplicates().set_index(["centre_id", "attacking_team"]).player

# df_goal = df[((df.shot_outcome == "Goal") | (df.type == "Own Goal For"))
#                     & (df.index.get_level_values(1) == df.team)]

# df_shot_player.name = "Tireur"
# df = pd.merge(df, df_shot_player, left_index = True, right_index = True, how = "left")

# df.loc[df_goal.index, "But"] = 1
# df.loc[df_goal.index, "Buteur"] = df_goal.player


# df_centre_drop = df.loc[(L_drop), :]
# df.drop(df_centre_drop[df_centre_drop.But == 0].index, inplace = True)


# df = df.reset_index(level = 1).drop("level_0", axis = 1)


# mask_centre = df.pass_cross == 1

# df_loc_centre = pd.DataFrame(df.loc[mask_centre, "location"].tolist(),
#     index = df.loc[mask_centre].index)
# df.loc[mask_centre, "x"] = df_loc_centre[0]
# df.loc[mask_centre, "y"] = df_loc_centre[1]

# df_loc_centre_end = pd.DataFrame(df.loc[mask_centre, "pass_end_location"].tolist(),
#     index = df.loc[mask_centre].index)
# df.loc[mask_centre, "x_end"] = df_loc_centre_end[0]
# df.loc[mask_centre, "y_end"] = df_loc_centre_end[1]


# mask_shot = (df.attacking_team == df.team) & (df.type == "Shot")

# df_loc_shot = pd.DataFrame(df.loc[mask_shot, "location"].tolist(),
#     index = df.loc[mask_shot].index)
# df.loc[mask_shot, "x"] = df_loc_shot[0]
# df.loc[mask_shot, "y"] = df_loc_shot[1]

# df_loc_shot_end = pd.DataFrame(df.loc[mask_shot, "shot_end_location"].tolist(),
#     index = df.loc[mask_shot].index)
# df.loc[mask_shot, "x_end"] = df_loc_shot_end[0]
# df.loc[mask_shot, "y_end"] = df_loc_shot_end[1]

# df


In [107]:
df.loc[138, :]

,level_0,shot_type,shot_outcome,type,match_id,period,possession,location,team,pass_cross,pass_type,index,pass_end_location,minute,player,shot_end_location,pass_body_part
attacking_team,,,,,,,,,,,,,,,,,
Angers,23062,None,None,Pass,3896589,2,145,"[86.9, 0.1]",Angers,1.0,Throw-in,2558,"[113.3, 31.8]",69,Lillian Rao-Lisoa,None,None
Angers,23063,None,None,Ball Recovery,3896589,2,145,"[10.9, 53.9]",Laval,NaN,None,2559,None,69,Mamadou Samassa,None,None
Angers,23064,None,None,Carry,3896589,2,145,"[10.9, 53.9]",Laval,NaN,None,2560,None,69,Mamadou Samassa,None,None
Angers,23065,None,None,Pass,3896589,2,145,"[20.4, 47.7]",Laval,NaN,None,2561,"[68.2, 11.1]",69,Mamadou Samassa,None,Right Foot
Angers,23066,None,None,Ball Receipt*,3896589,2,145,"[68.2, 11.1]",Laval,NaN,None,2562,None,69,Mohamed Amine Cherni,None,None
Angers,23067,None,None,Pass,3896589,2,145,"[65.0, 10.0]",Laval,NaN,None,2563,"[75.7, 12.7]",69,Mohamed Amine Cherni,None,Head


In [89]:
df_centre_drop = df_centre.loc[(L_drop), :]
df_centre.drop(df_centre_drop[df_centre_drop.But == 0].index).loc[6544, :]

,level_0,shot_type,shot_outcome,type,match_id,period,possession,location,team,pass_cross,pass_type,index,pass_end_location,minute,player,shot_end_location,pass_body_part,But
attacking_team,,,,,,,,,,,,,,,,,,
Bordeaux,987564,None,None,Pass,3896863,2,148,"[117.0, 21.8]",Bordeaux,1.0,None,2303,"[115.3, 42.7]",74,Jacques Ekomie,None,Left Foot,0
Bordeaux,987565,None,None,Ball Receipt*,3896863,2,148,"[115.3, 42.7]",Bordeaux,NaN,None,2304,None,74,Žan Vipotnik,None,None,0
Bordeaux,987566,Open Play,Off T,Shot,3896863,2,148,"[115.3, 42.7, 2.22]",Bordeaux,NaN,None,2305,None,74,Žan Vipotnik,"[120.0, 48.9, 2.1]",None,0
Bordeaux,987567,None,None,Goal Keeper,3896863,2,148,"[1.1, 42.3]",FC Annecy,NaN,None,2306,None,74,Florian Escales,None,None,0
Bordeaux,987568,None,None,Pass,3896863,2,149,"[6.0, 37.6]",FC Annecy,NaN,Goal Kick,2307,"[74.9, 16.6]",74,Florian Escales,None,Right Foot,0
Bordeaux,987569,None,None,Ball Receipt*,3896863,2,149,"[75.7, 14.4]",FC Annecy,NaN,None,2308,None,74,Nordine Kandil,None,None,0


In [68]:
L_drop = (df_centre[df_centre.level_0.duplicated()].index.get_level_values(0) - 1).unique()
df_centre_drop = df_centre.loc[(L_drop), :]
df_centre.drop(axis = 0, index = df_centre_drop[df_centre_drop.But != 1].index).groupby(level = [0, 1])["pass_cross"].apply(sum).sort_values(ascending=False)

centre_id  attacking_team
9106       Rodez             1.0
1          AC Ajaccio        1.0
2          AC Ajaccio        1.0
3          Rodez             1.0
4          AC Ajaccio        1.0
                            ... 
12         AC Ajaccio        1.0
11         AC Ajaccio        1.0
10         AC Ajaccio        1.0
9          AC Ajaccio        1.0
8          AC Ajaccio        1.0
Name: pass_cross, Length: 8919, dtype: float64

In [47]:
x = 6542
df_centre.loc[[x, x + 1, x + 2], :]

level_0  shot_type shot_outcome           type  \
centre_id attacking_team                                                   
6542      Bordeaux         987557       None         None           Pass   
          Bordeaux         987558       None         None  Ball Receipt*   
          Bordeaux         987559       None         None          Carry   
          Bordeaux         987560       None         None           Pass   
          Bordeaux         987561       None         None  Ball Receipt*   
          Bordeaux         987562       None         None          Carry   
6543      Bordeaux         987560       None         None           Pass   
          Bordeaux         987561       None         None  Ball Receipt*   
          Bordeaux         987562       None         None          Carry   
          Bordeaux         987563       None         None       Pressure   
          Bordeaux         987564       None         None           Pass   
          Bordeaux         987565       None         None  Ball Receipt*   
6544      Bordeaux         987564       None         None           Pass   
          Bordeaux         987565       None         None  Ball Receipt*   
          Bordeaux         987566  Open Play        Off T           Shot   
          Bordeaux         987567       None         None    Goal Keeper   
          Bordeaux         987568       None         None           Pass   
          Bordeaux         987569       None         None  Ball Receipt*   

                          match_id  period  possession             location  \
centre_id attacking_team                                                      
6542      Bordeaux         3896863       2         148        [113.0, 14.3]   
          Bordeaux         3896863       2         148         [98.4, 66.9]   
          Bordeaux         3896863       2         148         [98.4, 66.9]   
          Bordeaux         3896863       2         148        [100.3, 66.9]   
          Bordeaux         3896863       2         148        [115.3, 20.3]   
          Bordeaux         3896863       2         148        [115.3, 20.3]   
6543      Bordeaux         3896863       2         148        [100.3, 66.9]   
          Bordeaux         3896863       2         148        [115.3, 20.3]   
          Bordeaux         3896863       2         148        [115.3, 20.3]   
          Bordeaux         3896863       2         148          [7.1, 56.9]   
          Bordeaux         3896863       2         148        [117.0, 21.8]   
          Bordeaux         3896863       2         148        [115.3, 42.7]   
6544      Bordeaux         3896863       2         148        [117.0, 21.8]   
          Bordeaux         3896863       2         148        [115.3, 42.7]   
          Bordeaux         3896863       2         148  [115.3, 42.7, 2.22]   
          Bordeaux         3896863       2         148          [1.1, 42.3]   
          Bordeaux         3896863       2         149          [6.0, 37.6]   
          Bordeaux         3896863       2         149         [75.7, 14.4]   

                               team  pass_cross  pass_type  index  \
centre_id attacking_team                                            
6542      Bordeaux         Bordeaux         1.0       None   2296   
          Bordeaux         Bordeaux         NaN       None   2297   
          Bordeaux         Bordeaux         NaN       None   2298   
          Bordeaux         Bordeaux         1.0       None   2299   
          Bordeaux         Bordeaux         NaN       None   2300   
          Bordeaux         Bordeaux         NaN       None   2301   
6543      Bordeaux         Bordeaux         1.0       None   2299   
          Bordeaux         Bordeaux         NaN       None   2300   
          Bordeaux         Bordeaux         NaN       None   2301   
          Bordeaux        FC Annecy         NaN       None   2302   
          Bordeaux         Bordeaux         1.0       None   2303   
          Bordeaux         

In [45]:
(df_centre[df_centre.level_0.duplicated()].index.get_level_values(0) - 1).unique().tolist()

[78,
 100,
 141,
 171,
 179,
 228,
 271,
 334,
 408,
 427,
 506,
 510,
 550,
 598,
 773,
 856,
 857,
 923,
 949,
 1010,
 1054,
 1065,
 1090,
 1095,
 1098,
 1239,
 1413,
 1461,
 1492,
 1763,
 1795,
 1822,
 1854,
 2027,
 2049,
 2050,
 2056,
 2071,
 2152,
 2218,
 2291,
 2327,
 2425,
 2448,
 2497,
 2559,
 2562,
 2609,
 2669,
 2677,
 2708,
 2730,
 2766,
 2805,
 2815,
 2831,
 2857,
 2931,
 3046,
 3075,
 3155,
 3209,
 3212,
 3306,
 3368,
 3383,
 3436,
 3615,
 3683,
 3696,
 3716,
 3777,
 3812,
 3827,
 3842,
 3901,
 3961,
 3967,
 4225,
 4258,
 4327,
 4338,
 4416,
 4434,
 4446,
 4495,
 4582,
 4596,
 4628,
 4721,
 4797,
 4812,
 4877,
 4924,
 5116,
 5119,
 5157,
 5190,
 5241,
 5292,
 5340,
 5344,
 5367,
 5387,
 5391,
 5398,
 5401,
 5452,
 5487,
 5493,
 5524,
 5587,
 5610,
 5615,
 5622,
 5714,
 5804,
 5872,
 5905,
 6009,
 6093,
 6131,
 6254,
 6320,
 6322,
 6530,
 6542,
 6543,
 6548,
 6588,
 6636,
 6644,
 6651,
 6665,
 6704,
 6747,
 6791,
 6824,
 6827,
 6873,
 6985,
 7184,
 7263,
 7298,
 7301,
 7312

In [44]:
x = 78
df_centre.loc[[x, x + 1], :]

level_0  shot_type shot_outcome           type  \
centre_id attacking_team                                                   
78        Saint-Étienne     11602       None         None           Pass   
          Saint-Étienne     11603       None         None  Ball Receipt*   
          Saint-Étienne     11604       None         None      Clearance   
          Saint-Étienne     11605       None         None  Ball Recovery   
          Saint-Étienne     11606       None         None          Carry   
          Saint-Étienne     11607       None         None           Pass   
79        Saint-Étienne     11607       None         None           Pass   
          Saint-Étienne     11608       None         None  Ball Receipt*   
          Saint-Étienne     11609       None         None     Miscontrol   
          Saint-Étienne     11610       None         None  Ball Recovery   
          Saint-Étienne     11611  Open Play        Saved           Shot   
          Saint-Étienne     11612       None         None    Goal Keeper   

                          match_id  period  possession           location  \
centre_id attacking_team                                                    
78        Saint-Étienne    3896586       1          80      [107.5, 63.5]   
          Saint-Étienne    3896586       1          80      [110.8, 38.6]   
          Saint-Étienne    3896586       1          80        [9.7, 37.6]   
          Saint-Étienne    3896586       1          80      [117.4, 66.0]   
          Saint-Étienne    3896586       1          80      [117.4, 66.0]   
          Saint-Étienne    3896586       1          80      [117.7, 65.8]   
79        Saint-Étienne    3896586       1          80      [117.7, 65.8]   
          Saint-Étienne    3896586       1          80      [105.8, 49.6]   
          Saint-Étienne    3896586       1          80      [105.8, 49.2]   
          Saint-Étienne    3896586       1          80       [94.6, 52.5]   
          Saint-Étienne    3896586       1          80  [95.0, 47.2, 0.0]   
          Saint-Étienne    3896586       1          81        [1.3, 38.0]   

                                   team  pass_cross pass_type  index  \
centre_id attacking_team                                               
78        Saint-Étienne   Saint-Étienne         1.0      None   1466   
          Saint-Étienne   Saint-Étienne         NaN      None   1467   
          Saint-Étienne   Grenoble Foot         NaN      None   1468   
          Saint-Étienne   Saint-Étienne         NaN      None   1469   
          Saint-Étienne   Saint-Étienne         NaN      None   1470   
          Saint-Étienne   Saint-Étienne         1.0      None   1471   
79        Saint-Étienne   Saint-Étienne         1.0      None   1471   
          Saint-Étienne   Saint-Étienne         NaN      None   1472   
          Saint-Étienne   Saint-Étienne         NaN      None   1473   
          Saint-Étienne   Saint-Étienne         NaN      None   1474   
          Saint-Étienne   Saint-Étienne         NaN      None   1475   
          Saint-Étienne   Grenoble Foot         NaN      None   1476   

                         pass_end_location  minute           player  \
centre_id attacking_team                                              
78        Saint-Étienne      [110.4, 42.5]      36   Mathieu Cafaro   
          Saint-Étienne               None      36   Ibrahima Wadji   
          Saint-Étienne               None      36   Mamadou Diarra   
          Saint-Étienne               None      36   Mathieu Cafaro   
          Saint-Étienne               None      36   Mathieu Cafaro   
          Saint-Étienne      [105.8, 49.6]      36   Mathieu Cafaro   
79        Saint-Étienne      [105.8, 49.6]      36   Mathieu Cafaro   
          Saint-Étienne               None      36  Ibrahim Sissoko   
          Saint-Étienne               None      36  Ibrahim Sissoko   
          Saint-Étienne               None      36     Lamine Fomba   
          Saint-Étien

In [48]:
pd.concat([pd.Series(dico_event_test["Nb_but"], name = "Nombre but"), df_event_test], axis = 1).mean(axis = 0)

Nombre but        242.75
Ball Receipt*     244.25
Duel               24.75
Shot              246.50
Carry              23.00
Pressure           48.00
Error               3.25
Block               8.25
Ball Recovery      18.25
Goal Keeper         7.75
Clearance          11.50
Foul Committed      2.50
Pass                8.50
Miscontrol          0.75
Dribbled Past       0.25
Dribble             0.25
Foul Won            0.75
dtype: float64